In [1]:
import pickle as pkl
import numpy as np
import wfdb
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [105]:
curr_aux = df.loc[0]['aux']

def dynamic_replace(x,curr_aux=curr_aux):
    print(curr_aux)
    if x != '':
        curr_aux = x
    return curr_aux

In [71]:
class Segment:
    def __init__(self, signal=1, label=1, normal_ratio=0):
        self.signal = signal
        self.label = label
        self.normal_ratio = normal_ratio


def create_index_df(desired_segment_len=3600, basic_arr_path="data/mit-bih-arrhythmia-database-1.0.0"):
    desired_segment_len = 3600
    basic_arr_path = "data/mit-bih-arrhythmia-database-1.0.0"
    arr_db = wfdb.get_record_list('mitdb')
    num_samples_in_record = 30 * 60 * 360
    segment_dict = defaultdict(list)
    segment_list = list()
    
    # for selection and sampling 
    segment_dict_ann = {}
    record_count = 0
    
    
    for _, record_id in enumerate(arr_db):
        record_path = os.path.join(basic_arr_path, str(record_id))

        ann = wfdb.rdann(record_path, 'atr', sampto=num_samples_in_record, return_label_elements=['description',
                                                                                                  'symbol', 'label_store'])
        df = pd.DataFrame({'description': ann.description, 'sample': ann.sample, 'symbol': ann.symbol,
                           'label_store': ann.label_store, 'aux': ann.aux_note})
        counter = 0
        reset_flag = True
        allowed_labels = ['Normal beat']
        allowed_symbols = ['N']

        normal_counter = 0
        for i in range(1, df.shape[0] - 1):
            curr_label, curr_sample, curr_symbol = df.loc[i, ['description', 'sample', 'symbol']]
            if curr_label == 'Normal beat':
                normal_counter += 1
#             if curr_symbol == 'N':
#                 normal_counter += 1
            if reset_flag:
                start_sample = curr_sample
                ann_num_start = i
                normal_counter = 0
                allowed_labels = ['Normal beat']
                allowed_symbols = ['N']
            next_label, next_sample, next_symbol = df.loc[i + 1, ['description', 'sample', 'symbol']]
            if curr_label == next_label or next_label in allowed_labels or len(allowed_labels) < 2:
#             if curr_label == next_symbol or next_symbol in allowed_symbols or len(allowed_symbols) < 2:

                if next_label not in allowed_labels:
                    allowed_labels.append(next_label)
                    allowed_symbols.append(next_symbol)
#                 if next_symbol not in allowed_symbols:
#                     allowed_labels.append(next_label)
#                     allowed_symbols.append(next_symbol)
                ann_num_end = i+1
                counter += next_sample - curr_sample
                reset_flag = False
                if counter > desired_segment_len:
                    segment_dict['record_id'].extend([record_id])
                    segment_dict['start_sample'].extend([start_sample])
                    segment_dict['label'].extend([curr_label])
                    counter = 0
                    reset_flag = True
                    signal = wfdb.rdsamp(record_path, sampfrom=start_sample, sampto=start_sample + 3600)[0][:, 0]
                    normal_ratio = normal_counter / (ann_num_end - ann_num_start)
                    
                    df.loc[ann_num_start:ann_num_end]['aux'].unique().shape[0]
                    return
                    
                    segment_list.append(Segment(signal, allowed_labels[-1], normal_ratio))
                    segment_dict_ann[record_count] = [record_id,  allowed_labels[-1], signal, normal_ratio,
                                                       allowed_symbols[-1]]
                    record_count = record_count + 1
            else:
                counter = 0
                normal_counter = 0
                reset_flag = True
                allowed_labels = ['Normal beat']
                allowed_symbols = ['N']


    #     return pd.DataFrame(segment_dict)
#     return segment_list
    return segment_dict_ann

In [111]:
# d_dict = create_index_df()

record_path = os.path.join(basic_arr_path, str(record_id))

ann = wfdb.rdann(record_path, 'atr', sampto=num_samples_in_record, return_label_elements=['description',
                                                                                          'symbol', 'label_store'])
df = pd.DataFrame({'description': ann.description, 'sample': ann.sample,  'symbol': ann.symbol,
                           'label_store': ann.label_store, 'aux': ann.aux_note})

# df.loc[1:4]['aux'].unique().shape[0]
curr_aux = df.loc[0]['aux']

def dynamic_replace(s):
    s_aux = s['aux']
    curr_aux = s_aux.loc[0]
    
    for idx,x in s_aux.items():
        if x != '':
            curr_aux = x
        s_aux[idx] = curr_aux    
        

df.assign(aux = lambda x: dynamic_replace(x))
# basic_arr_path = "data/mit-bih-arrhythmia-database-1.0.0"
# num_samples_in_record = 30 * 60 * 360
df['aux'].value_counts()

# arr_db = wfdb.get_record_list('mitdb')
# record_id = arr_db[0]
# record_path = os.path.join(basic_arr_path, str(record_id))

# ann = wfdb.rdann(record_path, 'atr', sampto=num_samples_in_record, return_label_elements=['description','symbol', 'label_store'])
# ann.symbol

/home/bar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [61]:
seg_df = pd.DataFrame.from_dict(d_dict, orient='index')
seg_df.rename(columns={0: 'record_id', 1:'label', 2:'signal', 3:'normal_ratio',4:'symbol'}, inplace= True)
#  

In [62]:
# seg_df.symbol.value_counts()
seg_df[seg_df.label ==  'Normal beat'].head()

,record_id,label,signal,normal_ratio,symbol
1,100,Normal beat,"[0.885, 0.935, 0.835, 0.525, 0.12, -0.23, -0.4...",0.923077,N
2,100,Normal beat,"[0.88, 0.76, 0.46, 0.07, -0.24, -0.425, -0.485...",0.923077,N
3,100,Normal beat,"[0.885, 0.88, 0.68, 0.325, -0.08, -0.355, -0.5...",0.923077,N
4,100,Normal beat,"[0.85, 0.815, 0.6, 0.28, -0.1, -0.37, -0.495, ...",0.923077,N
5,100,Normal beat,"[0.775, 0.83, 0.685, 0.36, -0.03, -0.345, -0.4...",0.923077,N


In [45]:
num_labels_dict = {
'Normal beat': 283, #N
'Left bundle branch block beat': 103, #L
'Atrial premature beat': 66, # A
'Atrial flutter': 20,
'Atrial fibrillation': 135,
'Pre-excitation (WPW)': 21,
'Premature ventricular contraction': 133, #V
'Ventricular bigeminy': 55,
'Ventricular trigeminy': 13,
'Ventricular tachycardia': 10,
'Idioventricular rhythm': 10,
'Ventricular flutter': 10,   #!
'Fusion of ventricular and normal beat': 11, #F
'Second-degree heart block': 10,
'Pacemaker rhythm': 45,
'Supraventricular tachyarrhythmia': 13,
'Right bundle branch block beat': 62, #'R'
                 }

num_labels_we_have = {
'N': 283, #'Normal beat'
'L': 103, #'Left bundle branch block beat'
'A': 66, #'Atrial premature beat':
'V': 133, #'Premature ventricular contraction'
'!': 10,   #'Ventricular flutter'
'F': 11, # 'Fusion of ventricular and normal beat'
'R': 62 # 'Right bundle branch block beat'
               }



In [63]:
import random

# x = seg_df.loc[seg_df[seg_df.label == 'Normal beat'].index]

def sample_per_sym(seg_df, sym, num):
    # filter
    label_df = seg_df.loc[seg_df[seg_df.symbol == sym].index]
    # sample
    print(f' len df: {len(label_df)}')
    sample_df = label_df.sample(n=num)
    signals = np.stack(sample_df.signal.values)
    return sample_df, signals


sample_df = []
signals = []
for sym, num in num_labels_we_have.items():
    print(f'label:{label}, num: {num}')
    sample_df_label, signals_label = sample_per_sym(seg_df, sym, num)
    sample_df.append(sample_df_label)
    signals.append(signals_label)
sample_df = pd.concat(sample_df)


label:Atrial premature beat, num: 283
 len df: 3437
label:Atrial premature beat, num: 103
 len df: 494
label:Atrial premature beat, num: 66
 len df: 224
label:Atrial premature beat, num: 133
 len df: 1252
label:Atrial premature beat, num: 10
 len df: 11
label:Atrial premature beat, num: 11
 len df: 40
label:Atrial premature beat, num: 62
 len df: 489


In [67]:
sample_df.symbol.value_counts()

N    283
V    133
L    103
A     66
R     62
F     11
!     10
Name: symbol, dtype: int64

In [68]:
sample_df.to_pickle("./sampled.pkl")